In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from recommenders import ClusteringBasedRecommender,ClusteringAndAprioriBasedRecommender, Data, Apriori
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering, OPTICS, Birch
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer

In [4]:
import random 
def test_error(epochs, recommender,data:Data, seed=196, sample_size=1000):
    random.seed(seed)
    rand_seeds = [random.randint(0, 1000) for _ in range(epochs)]
    avg_error = 0
    for i in range(epochs):
        test_data = data.test_data.sample(n=sample_size, random_state=rand_seeds[i])
        test_data['predicted_rating'] = test_data.apply(
            lambda row: recommender.predict(int(row['userId']), int(row['movieId'])), axis=1
            )
        avg_error += np.mean(np.abs(test_data['rating'] - test_data['predicted_rating']))
    return avg_error / epochs

In [3]:
'''testing the clustering based reccomender on different combinations'''
imputers = [SimpleImputer(strategy='mean'),
            SimpleImputer(strategy='median'),
            KNNImputer(n_neighbors=4, weights='uniform')]
preprocessors = [[], [MinMaxScaler()],
                 [StandardScaler()], [Normalizer()],
                 [StandardScaler()],
                 [MinMaxScaler(), Normalizer()]]
clusterers = [KMeans(n_clusters=20), 
              DBSCAN(eps=0.5, min_samples=5),
              AgglomerativeClustering(n_clusters=20),
              SpectralClustering(n_clusters=20),
              OPTICS(min_samples=5),
              Birch(n_clusters=10)]
errors = {}

for i,imputer in enumerate(tqdm(imputers)):
    for preprocessor in preprocessors:
        for clusterer in clusterers:
            data = Data('ml-latest-small', 
                        imputer=imputer,
                        preprocessors=preprocessor)

            recommender = ClusteringBasedRecommender(
                data=data.train_data_table_for_clustering_normalized,
                data_unnormalized=data.train_data_table_for_clustering,
                movie_genres=data.movie_genres,
                clusterer=clusterer
                )
            recommender.train()
            errors[(imputer, tuple(preprocessor), clusterer)] = test_error(2, recommender,data,2, sample_size=100)


  0%|          | 0/3 [00:00<?, ?it/s]

c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-p

In [4]:
print("average error when predicting ratings:")
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print(value, key)

average error when predicting ratings:
0.6525000000000001 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20))
0.6575 (SimpleImputer(), (), Birch(n_clusters=10))
0.6575 (SimpleImputer(), (StandardScaler(),), KMeans(n_clusters=20))
0.6575 (KNNImputer(n_neighbors=4), (MinMaxScaler(),), AgglomerativeClustering(n_clusters=20))
0.66 (SimpleImputer(strategy='median'), (StandardScaler(),), KMeans(n_clusters=20))
0.665 (KNNImputer(n_neighbors=4), (), KMeans(n_clusters=20))
0.6675 (SimpleImputer(), (), KMeans(n_clusters=20))
0.6699999999999999 (SimpleImputer(strategy='median'), (MinMaxScaler(),), AgglomerativeClustering(n_clusters=20))
0.67 (KNNImputer(n_neighbors=4), (), Birch(n_clusters=10))
0.6725000000000001 (SimpleImputer(strategy='median'), (), KMeans(n_clusters=20))
0.675 (SimpleImputer(strategy='median'), (), Birch(n_clusters=10))
0.6775 (SimpleImputer(), (MinMaxScaler(),), KMeans(n_clusters=20))
0.6775 (KNNImputer(n_neighbors=4), (), AgglomerativeClustering(n_

In [5]:
'''one of twhe best clustering options tried with assosciation rules apriori, tried with 
different confidences and supports to check which is th best '''
imputer = SimpleImputer(strategy='median')
preprocessor = []
clusterer = AgglomerativeClustering(n_clusters=20)
#clusterer =KMeans(n_clusters=20)
data = Data('ml-latest-small', 
                        imputer= imputer,
                        preprocessors=preprocessor)
errors = {}
supports = [0.0007,0.0008,0.0009,0.001,0.002]
confidences = [0.7,0.8,0.9]
for supp in tqdm(supports):
    for  conf in confidences:
        recommender = ClusteringAndAprioriBasedRecommender(
        data=data.train_data_table_for_clustering_normalized,
        data_unnormalized=data.train_data_table_for_clustering,
        movie_genres=data.movie_genres,
        clusterer=clusterer,min_support = supp,metric="confidence",min_threshold=conf)
        recommender.train()
        errors[(imputer, tuple(preprocessor), clusterer,"support{}".format(supp),"confidence{}".format(conf))] = test_error(2, recommender,data,2, sample_size=100)
       
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print("average error when predicting ratings:")
    print(value, key)

100%|██████████| 5/5 [1:35:23<00:00, 1144.63s/it]

average error when predicting ratings:
0.6375 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.0007', 'confidence0.7')
average error when predicting ratings:
0.6375 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.0008', 'confidence0.7')
average error when predicting ratings:
0.6375 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.0009', 'confidence0.7')
average error when predicting ratings:
0.6375 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.001', 'confidence0.7')
average error when predicting ratings:
0.6375 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.002', 'confidence0.7')
average error when predicting ratings:
0.64 (SimpleImputer(strategy='median'), (), AgglomerativeClustering(n_clusters=20), 'support0.0007', 'confidence0.8')
average error when predicting ratings:
0.64 (Simpl

In [6]:
'''another of the best clustering combinations tried with assosciation rules apriori, 
tried with different confidences and supports to see which is the best '''
imputer = SimpleImputer()
preprocessor = []
clusterer = KMeans(n_clusters=20)
data = Data('ml-latest-small', 
                        imputer= imputer,
                        preprocessors=preprocessor)
errors = {}
supports = [0.0007,0.0008,0.0009,0.001,0.002]
confidences = [0.7,0.8,0.9]
for supp in tqdm(supports):
    for  conf in confidences:
        recommender = ClusteringAndAprioriBasedRecommender(
        data=data.train_data_table_for_clustering_normalized,
        data_unnormalized=data.train_data_table_for_clustering,
        movie_genres=data.movie_genres,
        clusterer=clusterer,min_support = supp,metric="confidence",min_threshold=conf)
        recommender.train()
        errors[(imputer, tuple(preprocessor), clusterer,"support{}".format(supp),"confidence{}".format(conf))] = test_error(2, recommender,data,2, sample_size=100)
        
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print("average error when predicting ratings:")
    print(value, key)

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 20/20 [11:06<00:00, 33.31s/it]


average error when predicting ratings:


c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 20/20 [10:35<00:00, 31.76s/it]


average error when predicting ratings:


c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
 20%|██        | 1/5 [32:19<2:09:19, 1939.79s/it]

...

average error when predicting ratings:


c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 20/20 [01:50<00:00,  5.54s/it]


average error when predicting ratings:


c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 5/5 [1:32:47<00:00, 1113.42s/it]

average error when predicting ratings:
0.6325000000000001 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.001', 'confidence0.9')
0.6375 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.001', 'confidence0.7')
0.6375 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.001', 'confidence0.8')
0.645 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0007', 'confidence0.7')
0.645 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0007', 'confidence0.8')
0.6475 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0008', 'confidence0.7')
0.6475 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0009', 'confidence0.9')
0.6525000000000001 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0009', 'confidence0.7')
0.6525000000000001 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.002', 'confidence0.7')
0.655 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0008', 'confidence0.8')
0.655 (SimpleImputer(), (), KMeans(n_clusters=20), 'support0.0009', 'con

In [ ]:
'''on more of the best clustering options tried with assosciation rules apriori, tried with 
different confidences and supports to check which is th best '''
imputer = SimpleImputer()
preprocessor = []
clusterer = Birch(n_clusters=10)
#clusterer =KMeans(n_clusters=20)
data = Data('ml-latest-small', 
                        imputer= imputer,
                        preprocessors=preprocessor)
errors = {}
supports = [0.0009,0.001,0.002]
confidences = [0.7,0.8,0.9]
for supp in tqdm(supports):
    for  conf in confidences:
        recommender = ClusteringAndAprioriBasedRecommender(
        data=data.train_data_table_for_clustering_normalized,
        data_unnormalized=data.train_data_table_for_clustering,
        movie_genres=data.movie_genres,
        clusterer=clusterer,min_support = supp,metric="confidence",min_threshold=conf)
        recommender.train()
        errors[(imputer, tuple(preprocessor), clusterer,"support{}".format(supp),"confidence{}".format(conf))] = test_error(2, recommender,data,2, sample_size=100)
       
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print("average error when predicting ratings:")
    print(value, key)

In [3]:
'''determining the optimal number of clusters when we decided to use Kmeans() with no preprocessor and SipleImputer after the tests 
in the previous three  cells'''
imputer = SimpleImputer()
preprocessor = []
data = Data('ml-latest-small', 
                        imputer=imputer,
                        preprocessors=preprocessor)
errors = {}
for i in range(2,100,2):
            clusterer = KMeans(n_clusters=i)
            recommender = ClusteringBasedRecommender(
                data=data.train_data_table_for_clustering_normalized,
                data_unnormalized=data.train_data_table_for_clustering,
                movie_genres=data.movie_genres,
                clusterer=clusterer
                )
            recommender.train()
            errors[(imputer, tuple(preprocessor), clusterer)] = test_error(2, recommender,data,2, sample_size=100)
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print(value, key)

c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-p

0.6325000000000001 (SimpleImputer(), (), KMeans(n_clusters=28))
0.635 (SimpleImputer(), (), KMeans(n_clusters=34))
0.635 (SimpleImputer(), (), KMeans(n_clusters=60))
0.6375 (SimpleImputer(), (), KMeans(n_clusters=32))
0.6375 (SimpleImputer(), (), KMeans(n_clusters=56))
0.64 (SimpleImputer(), (), KMeans(n_clusters=36))
0.645 (SimpleImputer(), (), KMeans(n_clusters=18))
0.645 (SimpleImputer(), (), KMeans(n_clusters=24))
0.645 (SimpleImputer(), (), KMeans(n_clusters=30))
0.645 (SimpleImputer(), (), KMeans(n_clusters=62))
0.6475 (SimpleImputer(), (), KMeans(n_clusters=20))
0.6475 (SimpleImputer(), (), KMeans(n_clusters=22))
0.6475 (SimpleImputer(), (), KMeans(n_clusters=44))
0.6475 (SimpleImputer(), (), KMeans(n_clusters=48))
0.6475 (SimpleImputer(), (), KMeans(n_clusters=58))
0.65 (SimpleImputer(), (), KMeans(n_clusters=86))
0.6525 (SimpleImputer(), (), KMeans(n_clusters=50))
0.6525000000000001 (SimpleImputer(), (), KMeans(n_clusters=92))
0.655 (SimpleImputer(), (), KMeans(n_clusters=40))

In [4]:
'''The best clustering option that we decided upon is now combined with an optimal number of clusters for it, and the Apriori algorithm'''
imputer = SimpleImputer()
preprocessor = []
clusterer = KMeans(n_clusters=28)
data = Data('ml-latest-small', 
                        imputer= imputer,
                        preprocessors=preprocessor)
errors = {}
supports = [0.0007,0.0008,0.0009,0.001,0.002]
confidences = [0.7,0.8,0.9]
for supp in tqdm(supports):
    for  conf in confidences:
        recommender = ClusteringAndAprioriBasedRecommender(
        data=data.train_data_table_for_clustering_normalized,
        data_unnormalized=data.train_data_table_for_clustering,
        movie_genres=data.movie_genres,
        clusterer=clusterer,min_support = supp,metric="confidence",min_threshold=conf)
        recommender.train()
        errors[(imputer, tuple(preprocessor), clusterer,"support{}".format(supp),"confidence{}".format(conf))] = test_error(2, recommender,data,2, sample_size=100)
        
for key, value in sorted(errors.items(), key=lambda x: x[1]):
    print("average error when predicting ratings:")
    print(value, key)

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 28/28 [08:19<00:00, 17.86s/it]
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
100%|██████████| 28/28 [08:49<00:00, 18.92s/it]
c:\Users\annamiczke\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the 

average error when predicting ratings:
0.595 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.001', 'confidence0.7')
average error when predicting ratings:
0.6174999999999999 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.0009', 'confidence0.9')
average error when predicting ratings:
0.6275 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.002', 'confidence0.7')
average error when predicting ratings:
0.63 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.0009', 'confidence0.8')
average error when predicting ratings:
0.63 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.001', 'confidence0.8')
average error when predicting ratings:
0.6325000000000001 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.0008', 'confidence0.7')
average error when predicting ratings:
0.6325000000000001 (SimpleImputer(), (), KMeans(n_clusters=28), 'support0.0008', 'confidence0.8')
average error when predicting ratings:
0.6325000000000001 (SimpleImputer(), (), KMeans(n_clusters